### Auditing Data Quality 

In [1]:
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

In [2]:
FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

In [5]:
def audit_file(filename, fields):
    fieldtypes = {}
    for field in fields:
        fieldtypes[field] = set([])

    # YOUR CODE HERE
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        
        #skipping the extra metadata
        for i in range(3):
            next(reader)

        # processing file
        for row in reader:
            for field in fields:
                value = row[field]
                if value == 'NULL' or value == '':
                    fieldtypes[field].add(type(None))
                elif value.startswith('{'):
                    fieldtypes[field].add(list)
                else:
                    try:
                        value = int(value)
                        fieldtypes[field].add(int)
                    except ValueError:
                        try:
                            value = float(value)
                            fieldtypes[field].add(float)
                        except ValueError:
                            fieldtypes[field].add(str)

    return fieldtypes

In [6]:
fieldtypes = audit_file(CITIES, FIELDS)

In [7]:
fieldtypes

{'areaCode': {list, NoneType, str, int},
 'areaLand': {float, NoneType, list},
 'areaMetro': {float, NoneType, list},
 'areaUrban': {float, NoneType, list},
 'elevation': {float, NoneType, list},
 'governmentType_label': {list, NoneType, str},
 'homepage': {list, NoneType, str},
 'isPartOf_label': {list, NoneType, str},
 'maximumElevation': {float, NoneType, list},
 'minimumElevation': {float, NoneType},
 'name': {list, NoneType, str},
 'populationDensity': {float, NoneType, list},
 'populationTotal': {list, NoneType, int},
 'timeZone_label': {list, NoneType, str},
 'utcOffset': {str, int, float, list, NoneType},
 'wgs84_pos#lat': {float, list, NoneType},
 'wgs84_pos#long': {float, list, NoneType}}

### Fixing the Area 

In [24]:
def fix_area(area):

    if area == "NULL":
        return None
    elif area[0] == "{":
        l = area.strip("{}").split("|")
        # Get string without 0's
        l1, l2 = str(l[0]).replace("e+", "").replace("0", ""), str(l[1]).replace("e+", "").replace("0", "")
        # Compare length of non-zero "significant" digits
        # Then return original which has more as a float
        if len(l1) > len(l2):
            return float(l[0])
        else:
            return float(l[1])
    return float(area)

In [25]:
def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra matadata
        for i in range(3):
            l = next(reader)

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)

    return data

In [27]:
result = process_file(CITIES)

### Fixing Names

In [32]:
def fix_name(name):

    # YOUR CODE HERE
    if name.startswith('{'):
        nameList = name.replace('{','').replace('}','').split('|')
        return nameList
    elif name == 'NULL':
        return []
    else:
        return [name]

In [33]:
def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = next(reader)

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data

In [34]:
result = process_file(CITIES)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [35]:
for n in range(20):
    pprint.pprint(result[n]["name"])

'Kud'
'Kuju'
'Kumbhraj'
'Kumhari'
'{Kundagola|Kundgol ಕುಂದಗೋಳ}'
'Kunigal'
'Kunzer'
'{Kurduvadi|कुर्डुवाडी}'
'Kurgunta'
'Kurinjipadi'
'Kurud'
'Kushtagi'
'{Ladnun|लाडनूँ}'
'{Lahar|लहार}'
'Laharpur'
'Lakheri'
'Lakhipur'
'{Laksar|लक्सर}'
'Lalkuan'
'Lalsot'


### Crossfield Auditing 

In [36]:
def check_loc(point, lat, longi):
    # YOUR CODE HERE
    pointList = point.split(' ')
    if lat == pointList[0]:
        return longi == pointList[1]
    else:
        return False

In [40]:
def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        # skipping the extra metadata
        for i in range(3):
            l = next(reader)
        # processing file
        for line in reader:
            # calling your function to check the location
            result = check_loc(line["point"], line["wgs84_pos#lat"],
                               line["wgs84_pos#long"])
            if not result:
                print ("{}: {} != {} {}".format(line["name"], line["point"],
                                               line["wgs84_pos#lat"],
                                               line["wgs84_pos#long"]))
            data.append(line)

    return data

In [44]:
result = process_file(CITIES)
result

Madambakkam: 12.903848 80.15861 != 12.9038 80.1586
Malihabad: {26.92 80.72|26.94 80.72} != {26.92|26.94} 80.72
Malkapur: {20.52 76.12|20.535997 76.112408} != {20.52|20.536} {76.1124|76.12}
{Masaurhi|मसौढ़ी}: {25.21 85.02|25.35 85.03333333333333} != {25.21|25.35} {85.02|85.0333}
Musafirkhana: {26.37 81.8|26.45 81.8} != {26.37|26.45} 81.8
{Narwana|thumbnail}: {29.616667 76.1116667|29.616667 76.116667} != 29.6167 {76.1117|76.1167}
Naspur: {18.83 79.45|18.860402 79.484882} != {18.83|18.8604} {79.45|79.4849}
Navalpattu: {10.7375 78.76722222222222|10.7425 78.76666666666667} != {10.7375|10.7425} {78.7667|78.7672}
Nowshera: {33.04 74.14|34.13 74.8} != {33.04|34.13} {74.14|74.8}
{Palayam|Palayam Kanyakumari}: 8.483333333333333 76.91666666666667 != 8.48333 76.9167
Pansemal: {21.65 74.7|21.66 74.7} != {21.65|21.66} 74.7
{Patur|पातूर}: {20.45 76.93|20.46 76.94} != {20.45|20.46} {76.93|76.94}
Periyakulam: {10.07 77.33|10.12 77.55} != {10.07|10.12} {77.33|77.55}
{Pedne / पेडणें|Pernem}: 15.716944444

IndexError: list index out of range